# Clasificación de municipios (Similares)

Un municipio puede ser “similar” a otro según varias dimensiones, como puede ser su Índice global (aunque esta es demasiado general). En este caso se emplean los vectores de categorías (salud, educación, bienestar, economía…). Municipios similares serán los que tengan perfiles parecidos en estas dimensiones.

Los motivos son los siguientes:

- Están normalizadas (0–1).

- Resumen información real del municipio.

- Son robustas a ruido.

- Son interpretables (cuando veas un cluster sabrás por qué están juntos).

Así que el clustering se hará sobre estas 6 columnas.

In [1]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_distances

In [2]:
# Cargar el conjunto de datos
df = pd.read_csv("../website_data/df_indices_categorias.csv")

feature_cols = ["sanidad", "educacion", "transporte_conectividad", "economia_empleo", "bienestar", "ocio_cultura", 'servicios', 'tranquilidad']
X = df[feature_cols].copy()
municipios = df["municipio"]

print(X.head())

    sanidad  educacion  transporte_conectividad  economia_empleo  bienestar  \
0  0.000000   0.979201                 0.724337         0.536833   0.341189   
1  0.700159   0.556944                 0.000000         0.771675   0.508173   
2  0.150871   0.718761                 0.365968         0.420842   0.146520   
3  0.511035   0.628643                 0.147945         0.627774   0.823978   
4  0.449325   0.614816                 0.090893         0.467898   0.832984   

   ocio_cultura  servicios  tranquilidad  
0      0.125203   0.000000          68.0  
1      0.703002   0.816960        4946.0  
2      0.142819   0.177304         256.0  
3      0.659231   0.742179       10413.0  
4      0.728355   0.682687        3422.0  


## PCA

- No dependes de clusters.
- Cada municipio tendrá su propio conjunto único de similares,

In [3]:
# PCA (2 O 3 componentes funcionanmuy bien)
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

# Matriz de distancias
dist_matrix = cosine_distances(X_pca)

# Función para obtener los municipios más similares
k = 10  # Número de municipios similares a encontrar

similares = {}

for i, muni in enumerate(municipios):
    # ordenar por distancia (de menor a mayor)
    idx = dist_matrix[i].argsort()[1:k+1]
    similares[muni] = municipios.iloc[idx].tolist()

# Convertimos a tabla
df_similares = (
    pd.DataFrame.from_dict(similares, orient="index")
      .reset_index()
)

df_similares.columns = ["municipio", "sim1", "sim2", "sim3", "sim4", "sim5", "sim6", "sim7", "sim8", "sim9", "sim10"]

df_similares.to_csv("../website_data/municipios_similares.csv", index=False)
df_similares.head()

,municipio,sim1,sim2,sim3,sim4,sim5,sim6,sim7,sim8,sim9,sim10
0,La Acebeda,Puebla de la Sierra,La Hiruela,Valdelaguna,El Atazar,Robledillo de la Jara,Santos de la Humosa (Los),Berzosa del Lozoya,Navalagamella,Villamanrique de Tajo,Valdemaqueda
1,Ajalvir,Casarrubuelos,Navas del Rey,Robledo de Chavela,Pelayos de la Presa,La Cabrera,Navacerrada,Villamanta,Torrelaguna,Buitrago del Lozoya,Torrejón de Velasco
2,Alameda del Valle,Villamanrique de Tajo,Gascones,Garganta de los Montes,Brea de Tajo,Orusco de Tajuña,La Hiruela,Valdepiélagos,Valdemaqueda,Valdaracete,La Acebeda
3,El Álamo,Humanes de Madrid,Moralzarzal,Brunete,Alpedrete,Velilla de San Antonio,Villalbilla,El Escorial,Torrejón de la Calzada,Torrelodones,San Lorenzo de El Escorial
4,Aldea del Fresno,Valverde de Alcalá,Perales de Tajuña,Valdeavero,Lozoyuela-Navas-Sieteiglesias,Navalafuente,Rascafría,Cenicientos,El Vellón,Fresnedillas de la Oliva,Corpa


¿Por qué usar distancia coseno?

Porque es más estable cuando los datos están estandarizados.

Evita que un municipio con valores más grandes parezca “más lejano”

Captura muy bien el “perfil” de un municipio